In [20]:
import json
import sys
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler# authentication
from tweepy.streaming import StreamListener# twitter tweets listener
from textwrap import TextWrapper
from datetime import datetime
from elasticsearch import Elasticsearch
import pandas as pd
from pandas import DataFrame

In [2]:
#consumer key, consumer secret, access token, access secret.
ckey='o60rkojmCTkqCRGtBFLH11e31'
csecret='5XPpbS9c0G1I36q0sLIRsttZzo9or8y0qTYIYeJRKRFqgBxkG3'
atoken='1074135523214680064-A9hyQMxS1NHAp7e28GS1F0U73wri84'
asecret='GaaRBkSP8yZ5JP5Z1Qwv07oivHgvKFSMu639SirlEmonF'

In [3]:
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

In [ ]:
#without using elastic search(using only tweepy) 
class listener(StreamListener):

    def on_data(self, data):
        print(data)
        return(True)

    def on_error(self, status):
        print(status)



twitterStream = Stream(auth, listener())
twitterStream.filter(track=["USA"])

In [5]:
host = {"host" : "localhost", "port" : 9200}
es = Elasticsearch(hosts = [host])# Connect to the elastic cluster

In [9]:
es.indices.create(index='tweeting',ignore =400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'tweeting'}

In [11]:
es.indices.create(index='twitter_index',ignore =400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'twitter_index'}

In [10]:
class StreamApi(StreamListener):

    def on_data(self, data):
        json_data=json.loads(data)
        print(json_data)
        
        es.index(index="tweeting",
                doc_type="teet",
                body={"author": json_data["user"],
                       "date": json_data["created_at"],
                       "message": json_data["text"]
                       })
                #ignore=400
        return True
      # on failure
    def on_error(self, status):
        print(status)

In [ ]:
if __name__ == '__main__':
    listener = StreamApi()
    
    auth = OAuthHandler(ckey, csecret)
    auth.set_access_token(atoken, asecret)
    
    streamer  = Stream(auth, listener)
    streamer.filter(track=['one','two','three'])        

In [6]:
es.indices.exists(index="tweeting") 

True

In [18]:
for index in es.indices.get('*'):
    print (index)

.kibana_1
inspections
tweeting
twitter_index
twt_ind


In [7]:
first = es.search(index="tweeting", body={"from":0,"size":1,"query":{"match_phrase":{"message":"snow day"}}})
first# got 0 hits so no tweets on 'snow day'

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 0, 'max_score': None, 'hits': []}}

In [8]:
second = es.search(index="tweeting", body={"from":0,"size":1,"query":{"match_phrase":{"date":"Mon Dec 17 19:45:14 +0000 2018"}}})
second # 5 tweets on dec 17th and at that precise time

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 5,
  'max_score': 2.1918683,
  'hits': [{'_index': 'tweeting',
    '_type': 'teet',
    '_id': 'HhqzvWcB5XnUAWVlkXSP',
    '_score': 2.1918683,
    '_source': {'author': {'id': 2401508816,
      'id_str': '2401508816',
      'name': 'CLM Burundi',
      'screen_name': 'clmburundi',
      'location': 'BUJUMBURA, BURUNDI',
      'url': None,
      'description': 'CLM. CHRIST-CENTERED. CELL BASED. TRANSFORMATION.',
      'translator_type': 'none',
      'protected': False,
      'verified': False,
      'followers_count': 392,
      'friends_count': 38,
      'listed_count': 3,
      'favourites_count': 124,
      'statuses_count': 666,
      'created_at': 'Fri Mar 21 12:41:09 +0000 2014',
      'utc_offset': None,
      'time_zone': None,
      'geo_enabled': True,
      'lang': 'en',
      'contributors_enabled': False,
      'is_translator': False,
      'profile_ba

In [12]:
es.indices.delete(index='twitter_index')#deleting an index created for practising

{'acknowledged': True}

In [13]:
for index in es.indices.get('*'):
    print (index)

.kibana_1
inspections
tweeting
twt_ind


In [14]:
OLHS_Library=es.get(index="tweeting", doc_type='teet',id='LRqzvWcB5XnUAWVllXQO') # from kibana i got the id for a single document 
#and used it to retrieve a document

In [15]:
print (OLHS_Library['_source'])

{'author': {'id': 3020368912, 'id_str': '3020368912', 'name': 'OLHS Library', 'screen_name': 'OLHSLRC', 'location': 'Motherwell', 'url': 'http://www.ourladyshighschool.co.uk', 'description': 'What goes on in the Library, plus news, competitions, stories, resources, #infolit and all sorts of interesting stuff. Tweets mainly by Mrs Macfadyen.', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 867, 'friends_count': 1832, 'listed_count': 42, 'favourites_count': 716, 'statuses_count': 4926, 'created_at': 'Thu Feb 05 19:16:56 +0000 2015', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang': 'en', 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': '000000', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme15/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme15/bg.png', 'profile_background_tile': False, 'profile_link_color': '9266CC', 'profile_sideb

In [16]:
es.search(index='twt_ind',body={
        'query':{
            'bool':{#using booleanto combination of queries search with authors not using english language and have zero followers.
                'must_not':{
                    'match':{
                        'author.lang':'en'
                    }
                },
                "filter":{
                    "range":{
                        "author.followers_count":{
                            "lte":0
                        }
                    }
                }
            }
        }
    })
#filtered using REST API

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 1,
  'max_score': 0.0,
  'hits': [{'_index': 'twt_ind',
    '_type': 'twt',
    '_id': 'fRgXumcBVgY_VFLmkCXV',
    '_score': 0.0,
    '_source': {'author': {'id': 2461904382,
      'id_str': '2461904382',
      'name': 'Michael',
      'screen_name': 'miguel_lilith',
      'location': 'México',
      'url': None,
      'description': 'Deportes Fútbol',
      'translator_type': 'none',
      'protected': False,
      'verified': False,
      'followers_count': 0,
      'friends_count': 31,
      'listed_count': 0,
      'favourites_count': 9,
      'statuses_count': 16,
      'created_at': 'Thu Apr 24 16:38:04 +0000 2014',
      'utc_offset': None,
      'time_zone': None,
      'geo_enabled': False,
      'lang': 'es',
      'contributors_enabled': False,
      'is_translator': False,
      'profile_background_color': 'C0DEED',
      'profile_background_image_url': 

Trying to include top 5 tweets owners name and follower count information into panda dataframe from elastic search

In [17]:
data = es.search(index="tweeting", doc_type='teet', body={"query": {"match_all": {}}}, size=5, _source=('author.screen_name','author.followers_count'))

In [18]:
df=data['hits']['hits']
df

[{'_index': 'tweeting',
  '_type': 'teet',
  '_id': 'FxqzvWcB5XnUAWVlj3Q3',
  '_score': 1.0,
  '_source': {'author': {'screen_name': 'kaylacampion',
    'followers_count': 146}}},
 {'_index': 'tweeting',
  '_type': 'teet',
  '_id': 'GhqzvWcB5XnUAWVlkHRW',
  '_score': 1.0,
  '_source': {'author': {'screen_name': 'GecoTree', 'followers_count': 723}}},
 {'_index': 'tweeting',
  '_type': 'teet',
  '_id': 'IhqzvWcB5XnUAWVlknRx',
  '_score': 1.0,
  '_source': {'author': {'screen_name': 'sbthemoor',
    'followers_count': 1887}}},
 {'_index': 'tweeting',
  '_type': 'teet',
  '_id': 'JxqzvWcB5XnUAWVlk3SU',
  '_score': 1.0,
  '_source': {'author': {'screen_name': 'ZalUIbaorimi',
    'followers_count': 17973}}},
 {'_index': 'tweeting',
  '_type': 'teet',
  '_id': 'KhqzvWcB5XnUAWVllHRT',
  '_score': 1.0,
  '_source': {'author': {'screen_name': 'NerdsSocialClub',
    'followers_count': 2188}}}]

In [21]:
df1=pd.DataFrame.from_dict(df)
source=df1._source

In [22]:
pd.concat(map(pd.DataFrame.from_dict, source))

,author
followers_count,146
screen_name,kaylacampion
followers_count,723
screen_name,GecoTree
followers_count,1887
screen_name,sbthemoor
followers_count,17973
screen_name,ZalUIbaorimi
followers_count,2188
screen_name,NerdsSocialClub


references
https://www.linkedin.com/pulse/stream-twitter-data-elasticsearch-dashboard-kibana-python-tripathi/

https://realpython.com/twitter-sentiment-python-docker-elasticsearch-kibana/

https://www.rittmanmead.com/blog/2015/08/three-easy-ways-to-stream-twitter-data-into-elasticsearch/

http://nitin-panwar.github.io/Elasticsearch-tutorial-for-beginners-using-Python/

https://tryolabs.com/blog/2015/02/17/python-elasticsearch-first-steps/